In [ ]:
%run vis/xAPISG-noDataToFillVisualization.ipynb # notebook to create the visualisation with a message NoDataToFill
%run vis/helpersFunctions/OrderGraphValuesAndSelectDataByXaxis.ipynb
%run vis/helpersFunctions/barCharts.ipynb

Selectors :
   * xaxisCompletableProgress : dropdown value indicating the number of bar per chart (Alls, 5, 25, 50, 10 first, 10 last)
   * orderCompletableProgress : dropdown value indicating how to order xlabel : alphabetically or by y value (hight to low or low to hight)

In [ ]:
#constructing a dropdown to select how many data we want to see in visualisations for x axis
xaxisCompletableProgress=widgets.Dropdown(options=[('All datas',0),('5 per visualisation',5),('25 per visualisation',25),('50 per visualisation',50),('10 first',-1),('10 last',-2)],
          description='x axe',
          disabled=False)
#constructing a dropdown to order for alphabetic keys or for values
orderCompletableProgress=widgets.Dropdown(options=[('alphabetic',0),('value from highest to lowest',1),('value from lowest to highest',-1)],
          description='Order',
          disabled=False)

***vis_completables_progress* function**:
* Output : the widgets selector and the outputCompletableProgress in a list to be display in a VBox
* outputCompletableProgress : display a bar chart showing for each player the progress achieved in the different completables of the game as well as in the total game

In [ ]:
def vis_completables_progress():
    def display_completable_progress(change):
        with outputCompletableProgress:
            clear_output(wait=False)
            getProgressPlayerPerCompletable()
            clear_output(wait=True)
    xaxisCompletableProgress.observe(display_completable_progress,'value')
    orderCompletableProgress.observe(display_completable_progress,'value')
    display_completable_progress(None)
    return [xaxisCompletableProgress, orderCompletableProgress, outputCompletableProgressUpdate, outputCompletableProgress]

***getProgressPlayerPerCompletable* function**:
Output : Displays bar chart showing for each player the progress achieved in the different completables of the game as well as in the total game

In [ ]:
def getProgressPlayerPerCompletable():
    playersSelected=checkboxesPlayersSelected.get_selected_options()
    completableSelectedProgress=getSublistSelectedFromUnionListSelected(completable_progress_list, checkboxesCompletableSelected.get_selected_options())
    if len(completableSelectedProgress)>0 and len(playersSelected)>0:
        if xaxisCompletableProgress.value in [0,-1,-2]:
            nbgraph=1
        else:
            div=len(completableSelectedProgress)/xaxisCompletableProgress.value
            nbgraph=math.ceil(div)
        accordionCompletableProgress=widgets.Accordion(children=[widgets.Output() for i in range(nbgraph)],
                                         selected_index=None)
        if nbgraph>1:
            for i in range(nbgraph):
                accordionCompletableProgress.set_title(i, str(i+1)+"/"+str(nbgraph))
        display(accordionCompletableProgress)
        def on_selected_index_completable_progress_change(change):
            n=change['new']
            if n!=None:
                #close and clear previous fig of completable progress
                for fig in figCompletableProgress:
                    plt.close(fig=fig)
                figCompletableProgress.clear()
                #get all players and completables selected
                playersSelected=checkboxesPlayersSelected.get_selected_options()
                completableSelectedProgress=getSublistSelectedFromUnionListSelected(completable_progress_list, checkboxesCompletableSelected.get_selected_options())
                # get progress values
                completab={}
                for player in playersSelected:
                    if not player in completab.keys():
                        completab[player]={}
                    if players_info[player]["completables_progress"] != {}:
                        for c in completableSelectedProgress:
                            if c in players_info[player]["completables_progress"].keys():
                                completab[player][c]=players_info[player]["completables_progress"][c]
                            else:
                                completab[player][c]=0
                # progress of each player per completable
                players_progress = {}
                for player in completab.keys():
                    for c in completableSelectedProgress:
                        if not c in players_progress.keys():
                            players_progress[c] = {}
                        if c in completab[player].keys():
                            players_progress[c][player] = completab[player][c]*100
                        else:
                            players_progress[c][player] = 0
                #order alls values
                ordered_progress = orderDictByDictValues(players_progress,orderCompletableProgress.value)
                #get positions of bar dict
                pos, bar_width = getPosDictMultiBarChart(ordered_progress,playersSelected)
                #separe in sub dict in function of xaxis dropout selected
                nbgraphshown, players_progress,N = organizeDictPerxAxisSelected(completableSelectedProgress,
                                                                                xaxisCompletableProgress.value,
                                                                                players_progress)
                # separe positions of bars in sub dict in function of xaxis dropout selected
                pos = organizeDictPosPerDataxaxis(xaxisCompletableProgress.value,pos)
                with accordionCompletableProgress.children[n]:
                    #create new fig with custom name in the tab
                    name=os.path.splitext(filename)[0]+"_completable_progress"
                    if nbgraphshown>1:
                        name+="_"+str(n+1)+"_"+str(nbgraphshown)
                    fig = plt.figure(num=name, constrained_layout=True)
                    fig.canvas.header_visible = False
                    fig.canvas.layout.min_height = '400px'
                    figCompletableProgress.append(fig)
                    # separe in sub dict in function of xaxis dropout selected
                    players_progress[n] = reorganiseDictBySubKeys(players_progress[n])
                    # display multibar with vertical lines
                    constructMultiBarWithVerticalLines(players_progress,n,
                                                       playersSelected,
                                                       completableSelectedProgress,
                                                       pos, bar_width, N)
                    #display title and labels
                    plt.xlabel("Completable")
                    plt.ylabel("Progress (in %)")
                    plt.title("Progress of players per completable (in %)")
                    plt.show()
        accordionCompletableProgress.observe(on_selected_index_completable_progress_change, 'selected_index')
        accordionCompletableProgress.selected_index=0
    else:
        #create new fig with custom name in the tab
        name=os.path.splitext(filename)[0]+"_completable_progress"
        fig = plt.figure(num=name, constrained_layout=True)
        fig.canvas.header_visible = False
        fig.canvas.layout.min_height = '400px'
        figCompletableProgress.append(fig)
        noDataToFillVis(100, True)
        #display title and labels
        plt.xlabel("Completable")
        plt.ylabel("Progress (in %)")
        plt.title("Progress of players per completable (in %)")
        plt.show()
    with outputCompletableProgressUpdate:
        clear_output(wait=False)